In [ ]:
import csv
from getpass import getpass
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

def get_tweet_data(card):
    "Extract data from tweet data"
    username = card.find_element("xpath", './/span/span').text
    handle = card.find_element("xpath", './/span[contains(text(), "@")]').text
    try:
        postdate = card.find_element("xpath",'.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    comment = card.find_element("xpath", './/div[@data-testid = "tweetText"]').text
    reply_count = card.find_element("xpath", './/div[@data-testid = "reply"]').text
    retweet = card.find_element("xpath", './/div[@data-testid = "retweet"]').text
    like = card.find_element("xpath", './/div[@data-testid = "like"]').text
    
    tweet = (username, handle, postdate, comment, reply_count, retweet, like)
    return tweet

# Create instance for web driver
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

# Navigate to login screen
driver.get('https://www.twitter.com/login')
driver.maximize_window()
sleep(10)
username = driver.find_element("xpath",'//input[@name = "text"]')
username.send_keys('raj_rajsingh03')
username.send_keys(Keys.RETURN)
sleep(1)
mypassword = getpass()

password = driver.find_element("xpath", '//input[@name = "password"]')
password.send_keys(mypassword)
password.send_keys(Keys.RETURN)
sleep(5)
# find serach input & search for term
search_input = driver.find_element("xpath", '//input[@aria-label = "Search query"]')
search_term = input("Enter Search Term: ")
search_input.send_keys(search_term)
search_input.send_keys(Keys.RETURN)

#Navigate to the latest tweets
driver.find_element("link text", 'Latest').click()
sleep(5)

#get all tweets from the page
data = []
tweet_ids = []
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = driver.find_elements("xpath", '//div[@data-testid = "cellInnerDiv"]')    
    for card in page_cards[-10:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.append(tweet_id)
                data.append(tweet)
                
    scroll_attempt = 0
    while True:
        #check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            
            #end of scroll region
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(5)
        else: 
            last_position = curr_position
            break

# Saving the tweet data

In [ ]:
with open('monkeypox_tweets.csv', 'w', newline = '', encoding = 'utf-8') as f:
    header = ['Username', 'Handle', 'Timestamp', 'Comments', 'Likes', 'Retweets', 'Text']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)
    